In [2]:
import torch

### 1. Tensor

In [3]:
x = torch.ones(2, 2, requires_grad=True) # 设置True可以跟踪x的变化过程
print(x)
print(x.grad_fn)   ## 因为x是叶子节点，他本身不经过任何运算得到（而是通过创建得到）

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [6]:
y = x + 2
print(y)
print(y.grad_fn)    ## 此处的y是通过Add运算得到，所以grad_fn属性不为空
                    ## grad_fn是它的实际的变化过程

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [7]:
## 判断一个tensor是否为叶子节点

print(x.is_leaf, y.is_leaf)

True False


In [8]:
## 更复杂的操作

z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [11]:
## 查看/验证requires_grad的作用

a = torch.randn(2, 2)             # 默认情况下requires_grad为False
b = ((a * 3) / (a - 1))
print(b.requires_grad)

a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


### 2. 梯度

In [12]:
out.backward()   # out由mean求得，本身是标量，不需要指定求导变量
                 # 等价于：out.backward(torch.tensor(1.))
print(x.grad)    # 求出out相对于x的梯度 d(out)/d(x)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [16]:
### 每次反向传播，都要先将梯度置零，否则梯度的变化是累加的
print(x)
out2 = x.sum()
print(out2)
out2.backward()
print(x.grad)

out3 = x.sum()
x.grad.data.zero_()   # 先将梯度值置零
out3.backward()
print(x.grad)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor(4., grad_fn=<SumBackward0>)
tensor([[2., 2.],
        [2., 2.]])
tensor([[1., 1.],
        [1., 1.]])


In [17]:
### 在进行out.backward()时，如果out是标量，那么就不需要为backward()传入任何参数
### 否则，需要传入一个与out同size的Tensor()

In [18]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)


In [19]:
### 由于z不是标量，因此想要在z的基础上求其对x的导数（梯度），需要先将z变成一个标量
### 方法是， 直接传入一个同z同size的权重向量，用来进行加权求和，以得到一个标量

In [20]:
v = torch.tensor([
        [1.0, 0.1],
        [0.01, 0.001]], dtype=torch.float
)
z.backward(v)
print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])


In [21]:
### 中断记录tensor的变化过程
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2
with torch.no_grad():
    y2 = x ** 3
y3 = y1 + y2

print(x.requires_grad)
print(y1, y1.requires_grad)
print(y2, y2.requires_grad)
print(y3, y3.requires_grad)

True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


In [22]:
y3.backward()
print(x.grad)

tensor(2.)


In [23]:
### 如果想要修改tensor的数值，但是又不希望被autograd记录，可以直接对tensor.data操作

In [25]:
x = torch.ones(1, requires_grad=True)

print(x.requires_grad)      # True
print(x.data)
print(x.data.requires_grad) # False

y = 2 * x
x.data *= 100

y.backward()
print(x)
print(x.grad)

True
tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])
